# Solving Differential equations using Julia Package
Writing the SDE as
$$
dX_t = f(X_t,p,t)dt + g(X_t,p,t) dB_t,\quad  X_0 = x_0
$$
with drift `f` and diffusion `g`, we can use `DifferentialEquations.jl` to quickly solve the problem.  The `p` argument is for any hyperparameters that one may wish to study.


In [ ]:
using Plots; pyplot()
using Random
using LaTeXStrings
using Statistics
using Printf
using DifferentialEquations
using DifferentialEquations.EnsembleAnalysis

In [ ]:
default(xtickfont=font(14),  ytickfont=font(14), guidefont=font(14), 
    legendfontsize=12, lw=2,ms=8)

## Example: OU Process
For the OU process
\begin{align*}
f(x,p,t) &= -\alpha(t)(x-m(t))\\
g(x,p,t) & = \sigma(t)
\end{align*}

In [ ]:
α= t-> 1;
m =t-> 0;
σ = t-> 1;

f(x,p,t) = -α(t) * (x-m(t))
g(x,p,t) = σ(t)

tmax = 10;
Δt = 0.01;
x0 = 0.;

# define the problem
prob = SDEProblem(f,g, x0, (0., tmax));

In [ ]:
# solve with Euler-Maruyama
soln = solve(prob, EM(), dt=Δt);

In [ ]:
plot(soln)

## Solving as an Ensemble
Built in features in `DifferentialEquations` faciliate the solution of an ensemble of realizations, and makes use of threasd.

In [ ]:
# Define the ensemble problem
ensemble_prob=EnsembleProblem(prob);

In [ ]:
# run the ensemble with the given number of realizations
n_samples = 10^2;
ensemble_soln= solve(ensemble_prob, EM(), dt=Δt, 
    EnsembleThreads(), trajectories = n_samples)

In [ ]:
ensemble_soln.u[1]

## Summary statistics
`DifferentialEquations.EnsembleAnalysis` includes tools for summarizing ensemble data.  It defaults to a 95% CI about the mean.

In [ ]:
ensemble_summary = EnsembleSummary(ensemble_soln)
plot(ensemble_summary, labels="Middle 95%")
ensemble_summary = EnsembleSummary(ensemble_soln; 
    quantiles = [0.25, 0.75])
plot!(ensemble_summary, labels="Middle 50%",legend=true)
xlabel!(L"$t$")

Somtimes it is helpful to manually construct the figure

In [ ]:
plot(ensemble_summary.t, ensemble_summary.u, 
    ribbon =sqrt.(ensemble_summary.v), label="95% CI")
xlabel!(L"$t$")